In [1]:
import pandas as pd
from sklearn import tree
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import random
import numpy as np

### Data Preparation and Clean up

In [2]:
file = "US_births(2018).csv"

In [3]:
# Read our Data file with the pandas library

file_df = pd.read_csv(file, encoding="ISO-8859-1")

C:\Users\Mercy\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
file_df.head()
#file_df.shape

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,...,0,1,2,190,1,2,N,0,M,41
1,1,1,33.3,2,3242,99,9999,2,1,1704,...,0,2,0,188,4,2,Y,2,F,0
2,1,1,30.0,0,3470,4,2017,1,1,336,...,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,...,0,2,0,138,1,2,N,0,F,0
4,1,1,35.5,0,2125,99,9999,1,1,830,...,0,1,4,220,3,1,N,0,M,0


In [5]:
file_df = file_df.dropna()

In [6]:
# Limit data to first time deliveries
delivery_method = file_df[file_df['RDMETH_REC'].isin(['1','2','3','4','5','6']) ]
delivery_method.head()

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,...,0,1,2,190,1,2,N,0,M,41
1,1,1,33.3,2,3242,99,9999,2,1,1704,...,0,2,0,188,4,2,Y,2,F,0
2,1,1,30.0,0,3470,4,2017,1,1,336,...,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,...,0,2,0,138,1,2,N,0,F,0
4,1,1,35.5,0,2125,99,9999,1,1,830,...,0,1,4,220,3,1,N,0,M,0


In [7]:
# print number of rows and columns in dataset
delivery_method.shape

(3799906, 55)

In [8]:
# Update delivery method to have only Cesareans and Vaginal birth
# delivery_method = delivery_method["RDMETH_REC"].replace({2:1, 4:3, 5:1, 6:3}, inplace=True)
delivery_method['RDMETH_REC'] = np.where((delivery_method.RDMETH_REC == 2),1,delivery_method.RDMETH_REC)
delivery_method['RDMETH_REC'] = np.where((delivery_method.RDMETH_REC == 4),3,delivery_method.RDMETH_REC)
delivery_method['RDMETH_REC'] = np.where((delivery_method.RDMETH_REC == 5),1,delivery_method.RDMETH_REC)
delivery_method['RDMETH_REC'] = np.where((delivery_method.RDMETH_REC == 6),3,delivery_method.RDMETH_REC)

In [9]:
delivery_method.head()

,ATTEND,BFACIL,BMI,CIG_0,DBWT,DLMP_MM,DLMP_YY,DMAR,DOB_MM,DOB_TT,...,PRIORDEAD,PRIORLIVE,PRIORTERM,PWgt_R,RDMETH_REC,RESTATUS,RF_CESAR,RF_CESARN,SEX,WTGAIN
0,1,1,30.7,0,3657,4,2017,1,1,1227,...,0,1,2,190,1,2,N,0,M,41
1,1,1,33.3,2,3242,99,9999,2,1,1704,...,0,2,0,188,3,2,Y,2,F,0
2,1,1,30.0,0,3470,4,2017,1,1,336,...,0,1,0,215,1,1,N,0,M,58
3,3,1,23.7,0,3140,5,2017,2,1,938,...,0,2,0,138,1,2,N,0,F,0
4,1,1,35.5,0,2125,99,9999,1,1,830,...,0,1,4,220,3,1,N,0,M,0


In [10]:
# selecting only needed columns into a dataframe
delivery_method = delivery_method[['RDMETH_REC','ATTEND','CIG_0','BMI','DMAR','LD_INDL','MAGER','M_Ht_In','NO_INFEC','NO_RISKS'
                                    ,'NO_MMORB','PWgt_R','WTGAIN','RF_CESAR','RF_CESARN','PRIORDEAD','PRIORLIVE','PRIORTERM']]
delivery_method.head()

,RDMETH_REC,ATTEND,CIG_0,BMI,DMAR,LD_INDL,MAGER,M_Ht_In,NO_INFEC,NO_RISKS,NO_MMORB,PWgt_R,WTGAIN,RF_CESAR,RF_CESARN,PRIORDEAD,PRIORLIVE,PRIORTERM
0,1,1,0,30.7,1,N,30,66,1,1,1,190,41,N,0,0,1,2
1,3,1,2,33.3,2,N,35,63,1,0,1,188,0,Y,2,0,2,0
2,1,1,0,30.0,1,N,28,71,1,0,1,215,58,N,0,0,1,0
3,1,3,0,23.7,2,N,23,64,1,1,1,138,0,N,0,0,2,0
4,3,1,0,35.5,1,N,37,66,1,1,1,220,0,N,0,0,1,4


In [11]:
# renaming columns
new_cols =['Delivery_Method','Medical_Attendant','Cigarette_Consumption','BMI','Marital_Status','Induction_of_labor'
               ,'Age_of_Mother','Mothers_Height_in_Inches','NO_INFEC','NO_RISK','NO_MORBIDITY','Pre_Pregnancy_Weight'
               ,'Weight_Gain','Previous_Cesarian','#_of_Previous_Cesareans','Prior_Births_Dead','Prior_Births_Living'
               ,'Prior_Terminations']

delivery_method.columns = new_cols

delivery_method.head()

,Delivery_Method,Medical_Attendant,Cigarette_Consumption,BMI,Marital_Status,Induction_of_labor,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,Weight_Gain,Previous_Cesarian,#_of_Previous_Cesareans,Prior_Births_Dead,Prior_Births_Living,Prior_Terminations
0,1,1,0,30.7,1,N,30,66,1,1,1,190,41,N,0,0,1,2
1,3,1,2,33.3,2,N,35,63,1,0,1,188,0,Y,2,0,2,0
2,1,1,0,30.0,1,N,28,71,1,0,1,215,58,N,0,0,1,0
3,1,3,0,23.7,2,N,23,64,1,1,1,138,0,N,0,0,2,0
4,3,1,0,35.5,1,N,37,66,1,1,1,220,0,N,0,0,1,4


In [12]:
# print number of columns in new data set
delivery_method.shape

(3799906, 18)

In [13]:
# get random dataset
delivery_method_df = delivery_method.sample(n=100000)
delivery_method_df.tail()

,Delivery_Method,Medical_Attendant,Cigarette_Consumption,BMI,Marital_Status,Induction_of_labor,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,Weight_Gain,Previous_Cesarian,#_of_Previous_Cesareans,Prior_Births_Dead,Prior_Births_Living,Prior_Terminations
1994282,1,1,0,16.3,2,N,22,65,1,1,1,98,44,N,0,0,0,0
867198,1,1,0,28.3,1,N,29,62,1,1,1,155,46,N,0,0,0,0
1353354,1,1,0,21.9,1,N,30,62,1,1,1,120,40,N,0,0,1,0
3800173,1,1,0,26.3,1,N,23,65,1,1,1,158,32,N,0,0,0,0
855708,1,1,0,28.3,1,N,25,63,0,1,1,160,20,N,0,0,0,0


In [14]:
delivery_method_df.to_csv('delivery_prediction.csv')

In [15]:
delivery_method_df.head()

,Delivery_Method,Medical_Attendant,Cigarette_Consumption,BMI,Marital_Status,Induction_of_labor,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,Weight_Gain,Previous_Cesarian,#_of_Previous_Cesareans,Prior_Births_Dead,Prior_Births_Living,Prior_Terminations
1734005,1,1,0,26.7,2,N,20,62,1,1,1,146,22,N,0,0,0,0
186609,3,1,1,19.3,,N,22,63,1,1,1,109,25,N,0,0,0,0
2023554,1,3,0,43.6,1,Y,23,66,1,0,1,270,21,N,0,0,0,0
3394473,1,3,0,27.4,1,N,34,68,1,1,1,180,42,N,0,0,5,1
397479,1,1,0,24.9,,N,22,64,1,1,1,145,37,N,0,0,1,0


In [16]:
delivery_method_df.dtypes

Delivery_Method               int64
Medical_Attendant             int64
Cigarette_Consumption         int64
BMI                         float64
Marital_Status               object
Induction_of_labor           object
Age_of_Mother                 int64
Mothers_Height_in_Inches      int64
NO_INFEC                      int64
NO_RISK                       int64
NO_MORBIDITY                  int64
Pre_Pregnancy_Weight          int64
Weight_Gain                   int64
Previous_Cesarian            object
#_of_Previous_Cesareans       int64
Prior_Births_Dead             int64
Prior_Births_Living           int64
Prior_Terminations            int64
dtype: object

In [17]:
#used get_dummies to Convert categorical variable into dummy/indicator variables.

delivery_method_df=pd.get_dummies(delivery_method_df)


In [18]:
delivery_method_df.head()

,Delivery_Method,Medical_Attendant,Cigarette_Consumption,BMI,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,...,Marital_Status_2,Marital_Status_,Marital_Status_1,Marital_Status_2,Induction_of_labor_N,Induction_of_labor_U,Induction_of_labor_Y,Previous_Cesarian_N,Previous_Cesarian_U,Previous_Cesarian_Y
1734005,1,1,0,26.7,20,62,1,1,1,146,...,1,0,0,0,1,0,0,1,0,0
186609,3,1,1,19.3,22,63,1,1,1,109,...,0,1,0,0,1,0,0,1,0,0
2023554,1,3,0,43.6,23,66,1,0,1,270,...,0,0,0,0,0,0,1,1,0,0
3394473,1,3,0,27.4,34,68,1,1,1,180,...,0,0,0,0,1,0,0,1,0,0
397479,1,1,0,24.9,22,64,1,1,1,145,...,0,1,0,0,1,0,0,1,0,0


### Split data into features and labels/Targets

In [19]:
# define target Column
target=delivery_method_df['Delivery_Method']
target_names=["vaginal","c-section"]


In [20]:
# Remove the labels from the features
# axis 1 refers to the columns
data=delivery_method_df.drop('Delivery_Method',axis=1)
# Saving feature names for later use
feature_names=data.columns
data.head()

,Medical_Attendant,Cigarette_Consumption,BMI,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,Weight_Gain,...,Marital_Status_2,Marital_Status_,Marital_Status_1,Marital_Status_2,Induction_of_labor_N,Induction_of_labor_U,Induction_of_labor_Y,Previous_Cesarian_N,Previous_Cesarian_U,Previous_Cesarian_Y
1734005,1,0,26.7,20,62,1,1,1,146,22,...,1,0,0,0,1,0,0,1,0,0
186609,1,1,19.3,22,63,1,1,1,109,25,...,0,1,0,0,1,0,0,1,0,0
2023554,3,0,43.6,23,66,1,0,1,270,21,...,0,0,0,0,0,0,1,1,0,0
3394473,3,0,27.4,34,68,1,1,1,180,42,...,0,0,0,0,1,0,0,1,0,0
397479,1,0,24.9,22,64,1,1,1,145,37,...,0,1,0,0,1,0,0,1,0,0


### Training and Testing Sets

In [21]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(data,target,random_state = 42)

In [22]:
X_train

,Medical_Attendant,Cigarette_Consumption,BMI,Age_of_Mother,Mothers_Height_in_Inches,NO_INFEC,NO_RISK,NO_MORBIDITY,Pre_Pregnancy_Weight,Weight_Gain,...,Marital_Status_2,Marital_Status_,Marital_Status_1,Marital_Status_2,Induction_of_labor_N,Induction_of_labor_U,Induction_of_labor_Y,Previous_Cesarian_N,Previous_Cesarian_U,Previous_Cesarian_Y
151699,1,0,31.9,29,63,1,0,1,180,41,...,1,0,0,0,1,0,0,0,0,1
1253693,1,0,31.2,26,63,1,0,1,176,10,...,0,0,0,0,1,0,0,0,0,1
985585,3,0,24.2,30,66,1,1,1,150,49,...,1,0,0,0,1,0,0,1,0,0
1634832,3,0,39.2,40,67,1,0,1,250,12,...,0,0,0,0,0,0,1,1,0,0
2460083,1,0,30.1,35,63,1,0,0,170,29,...,1,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066309,1,0,23.4,27,60,1,1,1,120,30,...,0,0,0,0,1,0,0,1,0,0
3331870,1,0,28.5,32,62,1,0,1,156,92,...,0,0,0,0,1,0,0,0,0,1
2696480,3,0,19.8,33,68,1,1,1,130,36,...,0,0,0,0,1,0,0,1,0,0
134864,3,0,17.8,29,66,1,0,1,110,35,...,1,0,0,0,1,0,0,1,0,0


In [23]:
# We expect the training features number of columns to match the testing feature number of columns
# and the number of rows to match for the respective training and testing features and the labels
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (75000, 25)
Training Labels Shape: (75000,)
Testing Features Shape: (25000, 25)
Testing Labels Shape: (25000,)


### Run Score tests to select the model

In [24]:
#Import Algorithm to use
from sklearn.ensemble import RandomForestClassifier
#Create RF classifier
rf = RandomForestClassifier(n_estimators=200)
# rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train random forest Classifier
rf = rf.fit(X_train,y_train)
# # Get the scores
rf.score(X_test,y_test)

0.80108

In [25]:
from sklearn.linear_model import LogisticRegression
#Create Logistic regression
cls = LogisticRegression()
# rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train random forest Classifier
cls = cls.fit(X_train,y_train)
# # Get the scores
cls.score(X_test,y_test)

# print(f"Training Data Score: {classifier.score(X_train, y_train)}")
# print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

C:\Users\Mercy\Anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7998

We perfomed 2 classifier tests on Random forest, logistic regression and Random forest had a slighter better out put
our data is classified, higher accuracy, handles the missing values and maintains the accuracy of a large proportion of data. If there are more trees, it won’t allow overfitting trees in the model. It has the power to handle a large data set with higher dimensionality

In [26]:
# In order to quantify the usefulness of all the variables in the entire random forest
importances = list(rf.feature_importances_)
sorted(zip(rf.feature_importances_,feature_names),reverse=True)

[(0.12535152622611342, 'Weight_Gain'),
 (0.12227461525840765, 'BMI'),
 (0.11142953151976706, 'Pre_Pregnancy_Weight'),
 (0.10510168112256031, 'Age_of_Mother'),
 (0.09403515510311916, 'Previous_Cesarian_Y'),
 (0.07053607796574969, 'Mothers_Height_in_Inches'),
 (0.06858947645719919, 'Previous_Cesarian_N'),
 (0.06604747995037097, '#_of_Previous_Cesareans'),
 (0.05392018740701743, 'Prior_Births_Living'),
 (0.045775753274078436, 'Medical_Attendant'),
 (0.03337048636752413, 'NO_RISK'),
 (0.02841222073921373, 'Prior_Terminations'),
 (0.015603283219968301, 'Cigarette_Consumption'),
 (0.011781739404903152, 'Induction_of_labor_Y'),
 (0.009833537414461252, 'Induction_of_labor_N'),
 (0.009239001602308488, 'Marital_Status_1'),
 (0.008769172788819092, 'Marital_Status_2'),
 (0.006200251184978796, 'Marital_Status_ '),
 (0.005114209592785073, 'NO_INFEC'),
 (0.0037386263382166132, 'NO_MORBIDITY'),
 (0.00301081342579876, 'Prior_Births_Dead'),
 (0.0009425907439030102, 'Marital_Status_1'),
 (0.0003385927202

### Make Predictions on the Test Set

In [27]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)


In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[15676,  1292],
       [ 3681,  4351]], dtype=int64)

### Improve Model if Necessary

In [44]:
# # create two datafames to provide equal samples to be teach the model
vaginal_df = delivery_method_df[delivery_method_df.Delivery_Method == 1]
#create a random sample of 32000 vaginal records
vaginal_df = vaginal_df.sample(n=32000)
csection_df = delivery_method_df[delivery_method_df.Delivery_Method == 3]
#create a random sample of 32000 section records
csection_df = csection_df.sample(n=32000)


(32000, 26)

array([3], dtype=int64)

### Interpret Model and Report Results

### Visualizing a Single Decision Tree

In [30]:
# One of the coolest parts of the Random Forest implementation in Skicit-learn is we can actually 
# examine any of the trees in the forest. 
# We will select one tree, and save the whole tree as an image
# The following code takes one tree from the forest and saves it as an image

In [31]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

ModuleNotFoundError: No module named 'pydot'

In [ ]:
# Pull out one tree from the forest
tree = rf.estimators_[5]

In [ ]:
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

In [ ]:
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

In [ ]:
# Write graph to a png file
graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(X_train, y_train)

In [ ]:
# Extract the small tree
tree_small = rf_small.estimators_[5]

In [ ]:
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)

In [ ]:
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png')

In [ ]:
### get only numeric features  by selecting only features that are not "object" datatypes
# numeric_variables=list(delivery_method_df.dtypes[delivery_method_df.dtypes != "object"].index)
# delivery_method_df[numeric_variables].head()

In [ ]:
#build first model
# model=RandomForestRegressor(n_estimators=100,oob_score=True, random_state=42)
# Fit the model
# model.fit(delivery_method_df[numeric_variables,Y ]


### Visualization

In [ ]:
# a simple bar plot of the feature importances 
# to illustrate the disparities in the relative significance of the variables
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')

# list of x locations for plotting
x_values = list(range(len(importances)))



In [ ]:
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [ ]:
# plot the entire dataset with predictions highlighted
# Use datetime for creating date objects for plotting
import datetime
# Dates of training values
months = features[:, feature_list.index('month')]
days = features[:, feature_list.index('day')]
years = features[:, feature_list.index('year')]
# List and then convert to datetime object
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]
# Dataframe with true values and dates
true_data = pd.DataFrame(data = {'date': dates, 'actual': labels})
# Dates of predictions
months = test_features[:, feature_list.index('month')]
days = test_features[:, feature_list.index('day')]
years = test_features[:, feature_list.index('year')]
# Column of dates
test_dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
# Convert to datetime objects
test_dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in test_dates]
# Dataframe with predictions and dates
predictions_data = pd.DataFrame(data = {'date': test_dates, 'prediction': predictions})
# Plot the actual values
plt.plot(true_data['date'], true_data['actual'], 'b-', label = 'actual')
# Plot the predicted values
plt.plot(predictions_data['date'], predictions_data['prediction'], 'ro', label = 'prediction')
plt.xticks(rotation = '60'); 
plt.legend()
# Graph labels
plt.xlabel('Date'); plt.ylabel('Maximum Temperature (F)'); plt.title('Actual and Predicted Values');


In [ ]:
# Make the data accessible for plotting
true_data['temp_1'] = features[:, feature_list.index('temp_1')]
true_data['average'] = features[:, feature_list.index('average')]
true_data['friend'] = features[:, feature_list.index('friend')]
# Plot all the data as lines
plt.plot(true_data['date'], true_data['actual'], 'b-', label  = 'actual', alpha = 1.0)
plt.plot(true_data['date'], true_data['temp_1'], 'y-', label  = 'temp_1', alpha = 1.0)
plt.plot(true_data['date'], true_data['average'], 'k-', label = 'average', alpha = 0.8)
plt.plot(true_data['date'], true_data['friend'], 'r-', label = 'friend', alpha = 0.3)
# Formatting plot
plt.legend(); plt.xticks(rotation = '60');
# Lables and title
plt.xlabel('Date'); plt.ylabel('Maximum Temperature (F)'); plt.title('Actual Max Temp and Variables');